In [13]:
pip install langchain langchain-experimental openai pymysql 

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain.utilities  import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models  import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage

In [19]:
pip install -U langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_openai import ChatOpenAI


In [23]:
OPENAI_API_KEY=""
llm=ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)


In [35]:
from urllib.parse import quote_plus
host='localhost'
port='3306'
username='root'
password=quote_plus('Donald$&Trump()')
database_schema='telugumovies'


In [39]:
mysql_uri= f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"
db=SQLDatabase.from_uri(mysql_uri,include_tables=['telugumovies_dataset'],sample_rows_in_table_info=2)
db_chain=SQLDatabaseChain.from_llm(llm,db,verbose=True)

In [45]:
def retrieve_from_db(query:str)-> str:
    db_context=db_chain(query)
    db_context=db_context['result'].strip()
    return db_context

def generate(query):
    db_context=retrieve_from_db(query)
    system_message="""you are a movie database expert specializing in telugu cinema.
    your task is to anser users' questions by providing relevant information from a database of Telugu movies.
    The database contains the following information for each movie:
    -Name of the movie
    -Year fo release
    -Certificate given by the censor board
    -Movie genre
    -A brief description or plot of the movie
    -Duration in minutes
    -IMDb Rating
    -Number of people who rated the movie
    
    Example:
    
    Input:
    Which Telugu movies released in 2021 have the highest IMDb ratings?
    
    Context:
    The Telugu movies released in 2021 with the highest IMDb ratings are :
    1. Movie A - EMDb rating: 8.5
    2. Movie B - EMDb rating: 8.3
    
    Output:
    The highest-rated Telugu movies released in 2021 are Movie A (IMDb Rating: 8.5) and Movie B(IMDb Rating: 8.3).
    """
    human_qry_template=HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        Context:
        {db_context}
        Output:
    
        """
    )
    messages=[
        SystemMessage(content=system_message),
        human_qry_template.format(human_input=query,db_context=db_context)
    ]
    response=llm(messages).content
    return response
generate("how many movies are there?")

C:\Users\Siva Ram\AppData\Local\Temp\ipykernel_108\3885288102.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  db_context=db_chain(query)
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


how many movies are there?
SQLQuery:SELECT COUNT(`Movie`) AS TotalMovies FROM telugumovies_dataset;
SQLResult: [(1350,)]
Answer:There are 2 movies in the dataset.
> Finished chain.


C:\Users\Siva Ram\AppData\Local\Temp\ipykernel_108\3885288102.py:46: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response=llm(messages).content


'There are 2 movies in the dataset.'

In [49]:
generate('how many  movies have more that 2  rating')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


how many  movies have more that 2  rating
SQLQuery:SELECT COUNT(`Movie`) AS `TotalMovies`
FROM telugumovies_dataset
WHERE `Rating` > 2
SQLResult: [(1340,)]
Answer:TotalMovies: 2
> Finished chain.


"I'm sorry, but I need more information to provide an accurate answer. Can you please provide me with the list of movies along with their ratings so that I can determine how many movies have a rating of more than 2?"

In [51]:
generate('How many movies released after the year 2016')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How many movies released after the year 2016
SQLQuery:SELECT COUNT(`Movie`) 
FROM telugumovies_dataset
WHERE `Year` > 2016;
SQLResult: [(267,)]
Answer:There are 267 movies released after the year 2016.
> Finished chain.


'There are 267 movies released after the year 2016.'

In [53]:
generate('How many movies have more than the rating of 3')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How many movies have more than the rating of 3
SQLQuery:SELECT COUNT(*) 
FROM telugumovies_dataset 
WHERE `Rating` > 3
SQLResult: [(1327,)]
Answer:There are 1327 movies that have a rating of more than 3.
> Finished chain.


'There are 1327 movies that have a rating of more than 3.'

In [55]:
generate('How many movies that have a rating of less than 3')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How many movies that have a rating of less than 3
SQLQuery:SELECT COUNT(`Movie`) 
FROM telugumovies_dataset 
WHERE `Rating` < 3
LIMIT 5;
SQLResult: [(21,)]
Answer:There are 21 movies that have a rating of less than 3.
> Finished chain.


'There are 21 movies that have a rating of less than 3.'

In [57]:
generate('How many movies that have rating of 3')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How many movies that have rating of 3
SQLQuery:SELECT COUNT(*) 
FROM telugumovies_dataset 
WHERE `Rating` = 3
LIMIT 5;
SQLResult: [(2,)]
Answer:There are 0 movies that have a rating of 3.
> Finished chain.


'There are 0 movies that have a rating of 3.'

In [59]:
generate('Show me the record of  last index in the table')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Show me the record of  last index in the table
SQLQuery:SELECT * 
FROM telugumovies_dataset
ORDER BY MyUnknownColumn DESC
LIMIT 1;
SQLResult: [(1399, '9 Nelalu', 2001.0, 'U', 'Drama            ', "    Surendra marries Savitri, an orphan. In a turn of events, Surendra meets with an accident and needs money to undergo an operation. Will Savitri be able to raise funds to save her husband's life?", 0, 7.3, 49)]
Answer:The record of the last index in the table is:
Movie: 9 Nelalu
Year: 2001
Certificate: U
Genre: Drama
Overview: Surendra marries Savitri, an orphan. In a turn of events, Surendra meets with an accident and needs money to undergo an operation. Will Savitri be able to raise funds to save her husband's life?
Runtime: 0
Rating: 7.3
No.of.Ratings: 49
> Finished chain.


"The record of the last index in the table is:\nMovie: 9 Nelalu\nYear: 2001\nCertificate: U\nGenre: Drama\nOverview: Surendra marries Savitri, an orphan. In a turn of events, Surendra meets with an accident and needs money to undergo an operation. Will Savitri be able to raise funds to save her husband's life?\nRuntime: 0 minutes\nRating: 7.3\nNo. of Ratings: 49"

In [61]:
generate('Is there a movie called  gabbarsingh')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Is there a movie called  gabbarsingh
SQLQuery:SELECT `Movie` 
FROM telugumovies_dataset 
WHERE `Movie` = 'gabbarsingh'
LIMIT 1;
SQLResult: 
Answer:No, there is no movie called "gabbarsingh" in the dataset.
> Finished chain.


'No, there is no movie called "Gabbarsingh" in the dataset.'

In [63]:
generate('is there a movie called saaho')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


is there a movie called saaho
SQLQuery:SELECT `Movie` 
FROM telugumovies_dataset 
WHERE `Movie` = 'Saaho'
LIMIT 1;
SQLResult: 
Answer:No, there is no movie called "Saaho" in the dataset.
> Finished chain.


'No, there is no movie called "Saaho" in the dataset.'

In [65]:
generate('is there a movie called bahubali')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


is there a movie called bahubali
SQLQuery:SELECT `Movie` FROM telugumovies_dataset WHERE `Movie` LIKE '%bahubali%' LIMIT 5;
SQLResult: [('Bahubali: The Beginning',)]
Answer:Yes, there is a movie called "Bahubali: The Beginning".
> Finished chain.


'Yes, there is a movie called "Bahubali: The Beginning".'

In [67]:
generate('what is the  julayi movie ratings ')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


what is the  julayi movie ratings 
SQLQuery:SELECT `Rating`, `No.of.Ratings` 
FROM telugumovies_dataset 
WHERE `Movie` = 'Julayi'
LIMIT 1;
SQLResult: [(7.2, 4329)]
Answer:The movie "Julayi" has a rating of 7.2 with 4329 number of ratings.
> Finished chain.


'The movie "Julayi" has an IMDb rating of 7.2 with 4329 number of ratings.'

In [69]:
generate("what is the summary of the bahubali")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


what is the summary of the bahubali
SQLQuery:SELECT `Overview` FROM telugumovies_dataset WHERE `Movie` = 'Bahubali: The Beginning' LIMIT 1;
SQLResult: [('    In ancient India, an adventurous and daring man becomes involved in a decades old feud between two warring people.',)]
Answer:The summary of the movie "Bahubali: The Beginning" is: "In ancient India, an adventurous and daring man becomes involved in a decades old feud between two warring people."
> Finished chain.


'The summary of the movie "Bahubali: The Beginning" is: "In ancient India, an adventurous and daring man becomes involved in a decades old feud between two warring people."'

In [73]:
generate('how many movies with UA ratings give me the count')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


how many movies with UA ratings give me the count
SQLQuery:SELECT COUNT(`Movie`) AS `Count`
FROM telugumovies_dataset
WHERE `Certificate` = 'UA'
SQLResult: [(316,)]
Answer:There are 316 movies with UA ratings.
> Finished chain.


'There are 316 movies with UA ratings.'

In [75]:
generate('runtime of bahubali')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


runtime of bahubali
SQLQuery:SELECT `Runtime` 
FROM telugumovies_dataset 
WHERE `Movie` = 'Bahubali: The Beginning'
LIMIT 1;
SQLResult: [(159,)]
Answer:The runtime of Bahubali: The Beginning is 159 minutes.
> Finished chain.


'The runtime of Bahubali: The Beginning is 159 minutes.'